Проверка комментариев к коду

# Построение модели определения стоимости автомобиля

**В данном проекте мы построим модель машинного обучения, способную наилучшим образом предсказать рыночную стоимость личных автомобилей клиентов.**
____________________________________________________________________________________
**Цель проекта:**

Построение модели машинного обучения, предсказывающей приближенную к реальности стоимость автомобиля клиента. Цель такой модели заключается в привлечении новых клиентов на сервис за счет новой полезной и интересной функции сайта.

В нашем распоряжении технические характеристики различных автомобилей и их стоимость, измеренная профессиональными оценщиками.

**Этапы выполнения проекта:**
1. Обзор данных.
2. Предобработка данных.
3. Анализ моделей.
4. Оценка эффективности моделей.

**Требования к модели:**
 - высокое качество предсказания;
 - высокая скорость предсказания, т.к. модель будет работать на сервисе в режиме онлайн;
 - высокое время обучения.

## Обзор данных

### Импорт данных

Установим и импортируем необходимые библиотеки.

In [1]:
# Импорт стандартных библиотек
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

# Импорт внешних библиотек
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [2]:
try:
    df = pd.read_csv('D:\\Users\\BlackEdition\\Desktop\\Data Science\\Обучение в Яндекс\\'
                     '12. Численные методы\\Проект\\autos.csv')
except:
    df = pd.read_csv('/datasets/autos.csv')

### Обзор данных

Посмотрим, что из себя представляют данные в имеющейся выборке.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
df.head(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


### Вывод

Из имеющихся данных видно, что:
 * датасет состоит из 354 369 строк;
 * в данных имеются пропуски;
 * формат даты необходимо преобразовать, если он будет использоваться;
 * всего мы имеем 15 фич, 6 из которых неинформативны для нашей будущей модели (*`DateCrawled`, `RegistrationMonth`, `DateCreated`,`NumberOfPictures`, `PostalCode`, `LastSeen`*);

Значение признаков:
 * `DateCrawled` — дата скачивания анкеты из базы
 * `VehicleType` — тип автомобильного кузова
 * `RegistrationYear` — год регистрации автомобиля
 * `Gearbox` — тип коробки передач
 * `Power` — мощность (л. с.)
 * `Model` — модель автомобиля
 * `Kilometer` — пробег (км)
 * `RegistrationMonth` — месяц регистрации автомобиля
 * `FuelType` — тип топлива
 * `Brand` — марка автомобиля
 * `NotRepaired` — была машина в ремонте или нет
 * `DateCreated` — дата создания анкеты
 * `NumberOfPictures` — количество фотографий автомобиля
 * `PostalCode` — почтовый индекс владельца анкеты (пользователя)
 * `LastSeen` — дата последней активности пользователя

## Предобработка данных

Проведем предварительную предобработку данных.

### Удаление неинформативных признаков

Удалим все неинформативные признаки, которые могу запутать модель или ухудшить метрики ее предсказания.

In [5]:
# Создание списка неинформативных признаков
uninformative_features = ['DateCrawled', 'RegistrationMonth', 'DateCreated',
                          'NumberOfPictures', 'PostalCode', 'LastSeen']

df = df.drop(uninformative_features, axis=1)

df.columns

Index(['Price', 'VehicleType', 'RegistrationYear', 'Gearbox', 'Power', 'Model',
       'Kilometer', 'FuelType', 'Brand', 'NotRepaired'],
      dtype='object')

### Обработка пропусков

Проверим данные на наличие NAN значений.

In [6]:
df.isna().sum()

Price                   0
VehicleType         37490
RegistrationYear        0
Gearbox             19833
Power                   0
Model               19705
Kilometer               0
FuelType            32895
Brand                   0
NotRepaired         71154
dtype: int64

В столбцах *`Gearbox`, `FuelType`, `NotRepaired`* имеются пропущенные значения, объем которых составляет до 20% от общего объема выборки. Удаление такого объема данных - критично. В связи с этим в столбцах *`Gearbox` и `FuelType`* мы заменим пропуски на наиболеее часто стречающиеся значения. В столбце *`NotRepaired`* укажем, что автомобили не битые, т.к. чаще всего ответ о том бита машина или нет - отрицательный. Пропуски в столбцах *`VehicleType`* и *`Model`* оставим без изменения, т.к. используемые нами модели самостоятельно определят такие пропуски в отдельную категорию, за исключением используемыех простых моделей.

In [7]:
# Замена NAN на моду
df['Gearbox'] = df['Gearbox'].fillna(df['Gearbox'].mode().values[0])
df['FuelType'] = df['FuelType'].fillna(df['FuelType'].mode().values[0])

# Замена NAN на 'no'
df['NotRepaired'] = df['NotRepaired'].fillna('no')

In [8]:
df.isna().sum()

Price                   0
VehicleType         37490
RegistrationYear        0
Gearbox                 0
Power                   0
Model               19705
Kilometer               0
FuelType                0
Brand                   0
NotRepaired             0
dtype: int64

Замена NAN значений прошла успешно.

### Обработка дубликатов

Найдем явные дубликаты в данных и удалим их.

In [9]:
df.duplicated().sum()

56547

In [10]:
df = df.drop_duplicates()

In [11]:
df.duplicated().sum()

0

Удаление дубликатов прошло успешно.

### Поиск и обработка аномалий

Найдем аномалии в данных.

In [12]:
'''Цикл, который выводит уникальные значения в столбцах,
отсортированные по ascending=True.
'''


for column in df.columns:
    anomaly = df[column].sort_values().unique()
    display(column)
    display(anomaly)
    display()

'Price'

array([    0,     1,     2, ..., 19998, 19999, 20000], dtype=int64)

'VehicleType'

array(['bus', 'convertible', 'coupe', 'other', 'sedan', 'small', 'suv',
       'wagon', nan], dtype=object)

'RegistrationYear'

...

array(['cng', 'electric', 'gasoline', 'hybrid', 'lpg', 'other', 'petrol'],
      dtype=object)

'Brand'

array(['alfa_romeo', 'audi', 'bmw', 'chevrolet', 'chrysler', 'citroen',
       'dacia', 'daewoo', 'daihatsu', 'fiat', 'ford', 'honda', 'hyundai',
       'jaguar', 'jeep', 'kia', 'lada', 'lancia', 'land_rover', 'mazda',
       'mercedes_benz', 'mini', 'mitsubishi', 'nissan', 'opel', 'peugeot',
       'porsche', 'renault', 'rover', 'saab', 'seat', 'skoda', 'smart',
       'sonstige_autos', 'subaru', 'suzuki', 'toyota', 'trabant',
       'volkswagen', 'volvo'], dtype=object)

'NotRepaired'

array(['no', 'yes'], dtype=object)

Аномалии видны в столбацах *`RegistrationYear`* и *`Power`*.
В столбце *`RegistrationYear`* имеются автомобили, поставленные на учет еще при существовании татарско-монгольского ига, а так же автомобили из будущего. Массовое конвеерно машиностроение началось с 1936 года и продолжается по сей день, поэтому в данных мы ограничимся этими годами.
В части мощности ограничимся 50-1000 л.с. т.к. машины меньшей и большей мощности в наше время редкость.

Избавимся от аномалий:

In [13]:
# Годы регистрации автомобиля от 1936 до 2022
df = df.loc[(df['RegistrationYear'] >= 1936) & (df['RegistrationYear'] <= 2016)]
# Мощность от 50 до 1000 л.с.
df = df.loc[(df['Power'] >= 50) & (df['Power'] <= 1000)]
# Цена от 100$
df = df.loc[df['Price'] >= 100]

display(df['RegistrationYear'].sort_values().unique())
display()
display(df['Power'].sort_values().unique())
display()
display(df.shape)

array([1937, 1941, 1942, 1943, 1945, 1947, 1948, 1949, 1950, 1951, 1952,
       1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963,
       1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974,
       1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985,
       1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996,
       1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
       2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016], dtype=int64)

array([  50,   51,   52,   53,   54,   55,   56,   57,   58,   59,   60,
         61,   62,   63,   64,   65,   66,   67,   68,   69,   70,   71,
         72,   73,   74,   75,   76,   77,   78,   79,   80,   81,   82,
         83,   84,   85,   86,   87,   88,   89,   90,   91,   92,   93,
         94,   95,   96,   97,   98,   99,  100,  101,  102,  103,  104,
        105,  106,  107,  108,  109,  110,  111,  112,  113,  114,  115,
        116,  117,  118,  119,  120,  121,  122,  123,  124,  125,  126,
        127,  128,  129,  130,  131,  132,  133,  134,  135,  136,  137,
        138,  139,  140,  141,  142,  143,  144,  145,  146,  147,  148,
        149,  150,  151,  152,  153,  154,  155,  156,  157,  158,  159,
        160,  161,  162,  163,  164,  165,  166,  167,  168,  169,  170,
        171,  172,  173,  174,  175,  176,  177,  178,  179,  180,  181,
        182,  183,  184,  185,  186,  187,  188,  189,  190,  191,  192,
        193,  194,  195,  196,  197,  198,  199,  2

(243164, 10)

Все аномалии удалены.

### Кодирование, масштабирование, разбивка данных

#### Для регрессий

**Разбивка данных на фичи и таргет**

In [14]:
features = df.drop('Price', axis=1)
target = df['Price']

**Кодирование категориальных фич**

In [15]:
features = pd.get_dummies(features, drop_first=True)

Разбивка на train и valid+test выборки.

In [16]:
(features_train, features_valid_test,
 target_train, target_valid_test) = train_test_split(features,
                                                     target,
                                                     test_size=0.4,
                                                     random_state=12345)

display(features_train.shape)
display(target_train.shape)
display(features_valid_test.shape)
display(target_valid_test.shape)

(145898, 305)

(145898,)

(97266, 305)

(97266,)

Valid+test выборку разобьем на valid и test по отдельности.

In [17]:
(features_valid, features_test,
 target_valid, target_test) = train_test_split(features_valid_test,
                                               target_valid_test,
                                               test_size=0.5,
                                               random_state=12345)

display(features_valid.shape)
display(target_valid.shape)
display(features_test.shape)
display(target_test.shape)

(48633, 305)

(48633,)

(48633, 305)

(48633,)

**Масштабирование данных**

In [18]:
# Масштабирование признаков
scale_list = ['RegistrationYear', 'Power', 'Kilometer']

scaler = StandardScaler()
scaler.fit(features_train[scale_list])

features_train[scale_list] = scaler.transform(features_train[scale_list])
features_valid[scale_list] = scaler.transform(features_valid[scale_list])
features_test[scale_list] = scaler.transform(features_test[scale_list])

display(features_train)
display(features_valid)
display(features_test)

C:\Users\BlackEdition\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\Users\BlackEdition\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\Users\BlackEdition\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

,RegistrationYear,Power,Kilometer,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_wagon,...,Brand_skoda,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_yes
238049,-0.943598,0.479550,0.612755,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
312458,0.759202,-0.071208,-1.253400,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
248846,1.068803,-0.695400,0.612755,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36636,-2.181999,-0.621966,-0.986807,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
267886,0.759202,0.479550,0.612755,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112189,-0.479198,0.351040,0.612755,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
180649,-0.014798,-1.264516,-2.053181,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
176363,0.140002,0.222529,-0.053729,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100785,0.759202,0.222529,-2.053181,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


,RegistrationYear,Power,Kilometer,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_wagon,...,Brand_skoda,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_yes
273791,0.914002,0.975232,-0.986807,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
264832,-1.098399,-0.530173,0.612755,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
213195,0.294802,-0.273153,0.612755,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
148723,0.294802,0.920156,-0.053729,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
44262,-1.098399,-1.172724,-3.252853,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29715,-0.788798,-0.934062,0.612755,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
192144,0.604402,-1.080931,-0.053729,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
241979,-0.479198,-0.897345,0.612755,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
214723,0.759202,0.295964,-1.253400,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


,RegistrationYear,Power,Kilometer,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_wagon,...,Brand_skoda,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_yes
330578,-0.169598,0.424474,-0.053729,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
333381,-1.098399,-0.144642,0.612755,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
191720,-0.479198,-0.493456,0.612755,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
243038,0.294802,-0.144642,-3.252853,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
83245,-0.788798,-0.621966,0.612755,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69622,0.294802,0.020585,-0.053729,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11165,0.294802,-0.236435,0.612755,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
98197,-0.633998,0.222529,0.612755,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
216710,-0.633998,-1.172724,0.612755,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


#### Для деревьев

**Разбика данных на фичи и таргет для деревьев**

In [19]:
df_tree = df.copy()

In [20]:
df_tree['VehicleType'] = df_tree['VehicleType'].fillna(df_tree['VehicleType']
                                               .mode()
                                               .values[0])
df_tree['Model'] = df_tree['Model'].fillna(df_tree['Model']
                                   .mode()
                                   .values[0])

In [21]:
# Разбивка данных на фичи и таргет
features_tree = df_tree.drop('Price', axis=1)
target_tree = df_tree['Price']

**Кодирование категориальных фич для деревьев**

In [22]:
encoder_list = ['VehicleType', 'Gearbox', 'Model',
                'FuelType', 'Brand', 'NotRepaired']
ordinal_encoder = OrdinalEncoder()
ordinal_encoder.fit(features_tree[encoder_list])
features_tree[encoder_list] = ordinal_encoder.transform(features_tree[encoder_list])
display(features_tree)

,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
1,2.0,2011,1.0,190,116.0,125000,2.0,1.0,1.0
2,6.0,2004,0.0,163,117.0,125000,2.0,14.0,0.0
3,5.0,2001,1.0,75,116.0,150000,6.0,38.0,0.0
4,5.0,2008,1.0,69,101.0,90000,2.0,31.0,0.0
5,4.0,1995,1.0,102,11.0,150000,6.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...
354361,4.0,2016,0.0,150,4.0,150000,6.0,0.0,0.0
354362,4.0,2004,1.0,225,140.0,150000,6.0,30.0,1.0
354366,1.0,2000,0.0,101,106.0,125000,6.0,32.0,0.0
354367,0.0,1996,1.0,102,223.0,150000,2.0,38.0,0.0


In [23]:
(features_train_tree, features_valid_tree,
 target_train_tree, target_valid_tree) = train_test_split(features_tree,
                                                          target_tree,
                                                          test_size=0.25,
                                                          random_state=12345)

display(features_train_tree.shape)
display(target_train_tree.shape)
display(features_valid_tree.shape)
display(target_valid_tree.shape)

(182373, 9)

(182373,)

(60791, 9)

(60791,)

In [24]:
(features_valid_tree, features_test_tree,
 target_valid_tree, target_test_tree) = train_test_split(features_valid_tree,
                                                         target_valid_tree,
                                                         test_size=0.5,
                                                         random_state=12345)

display(features_valid_tree.shape)
display(target_valid_tree.shape)
display(features_test_tree.shape)
display(target_test_tree.shape)

(30395, 9)

(30395,)

(30396, 9)

(30396,)

### Вывод

Нами произведена обработка данных для возможности их использования при обучении моделей.
Успешно выполнены:
 * удаление неинформативных признаков;
 * обработка пропусков, дубликатов и аномалий;
 * кодирование, мастабирование признаков и их разделение на обучающую и валидационную выборки для моделей градиентного бустинга;
 * кодирование, мастабирование признаков и их разделение на обучающую и валидационную выборки для деревьев.

## Анализ моделей

Обучим наши модели на обработанных данных и оценим их объективность через метрику  Root mean squeared error.

### DesicionTreeRegressor

**Вариант проверки № 1**

In [25]:
model_tree = DecisionTreeRegressor(max_depth=10, random_state=12345)

In [26]:
%%time

model_tree.fit(features_train_tree, target_train_tree)

Wall time: 589 ms


DecisionTreeRegressor(max_depth=10, random_state=12345)

In [27]:
%%time

predictions_valid = model_tree.predict(features_valid_tree)
rmse = mean_squared_error(target_valid_tree, predictions_valid) ** .5

display(rmse)

2031.0626018766036

Wall time: 14 ms


**Вариант проверки № 2**

In [28]:
model_tree_2 = DecisionTreeRegressor(max_depth=50, random_state=12345)

In [29]:
%%time

model_tree_2.fit(features_train_tree, target_train_tree)

Wall time: 1.05 s


DecisionTreeRegressor(max_depth=50, random_state=12345)

In [30]:
%%time

predictions_valid = model_tree_2.predict(features_valid_tree)
rmse = mean_squared_error(target_valid_tree, predictions_valid) ** .5

display(rmse)

2043.8760611906173

Wall time: 28.5 ms


### LGBMRegressor

**Вариант проверки № 1**

In [31]:
model_LGBM = LGBMRegressor(learning_rate=1,
                           max_depth=5,
                           metric='rmse',
                           random_state=12345)

In [32]:
%%time

model_LGBM.fit(features_train, target_train)

Wall time: 1.95 s


LGBMRegressor(learning_rate=1, max_depth=5, metric='rmse', random_state=12345)

In [33]:
%%time

predictions_valid = model_LGBM.predict(features_valid)
rmse = mean_squared_error(target_valid, predictions_valid) ** .5

display(rmse)

1747.7475893240712

Wall time: 418 ms


**Вариант проверки № 2**

In [34]:
model_LGBM_2 = LGBMRegressor(learning_rate=1,
                             max_depth=10,
                             metric='rmse',
                             random_state=12345)

In [35]:
%%time

model_LGBM_2.fit(features_train, target_train)

Wall time: 2 s


LGBMRegressor(learning_rate=1, max_depth=10, metric='rmse', random_state=12345)

In [36]:
%%time

predictions_valid = model_LGBM_2.predict(features_valid)
rmse = mean_squared_error(target_valid, predictions_valid) ** .5

display(rmse)

1720.709567310277

Wall time: 446 ms


### CatBoostRegressor

**Вариант проверки № 1**

In [37]:
model_CB = CatBoostRegressor(learning_rate=1,
                             depth=5,
                             loss_function='RMSE',
                             random_state=12345)

In [38]:
%%time

model_CB.fit(features_train,
             target_train,
             verbose=False,
             plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Wall time: 22.1 s


In [39]:
%%time

predictions_valid = model_CB.predict(features_valid)
rmse = mean_squared_error(target_valid, predictions_valid) ** .5
display(rmse)

1661.2740942754344

Wall time: 172 ms


**Вариант проверки № 2**

In [40]:
model_CB_2 = CatBoostRegressor(learning_rate=1,
                               depth=10,
                               loss_function='RMSE',
                               random_state=12345)

In [41]:
%%time

model_CB_2.fit(features_train, target_train, verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Wall time: 57.7 s


In [42]:
%%time

predictions_valid = model_CB_2.predict(features_valid)
rmse = mean_squared_error(target_valid, predictions_valid) ** .5
display(rmse)

1705.4461359939774

Wall time: 575 ms


### Вывод

По итогам проведенного обучения у нас имеются следующие показатели:

| Вариант реализации | Показатели | DecisionTreeRegressor | LGBMRegressor | CatBoostRegressor |
| ------------------ | -----------| --------------------- | ------------- | ----------------- |
| Вариант № 1 | RMSE | 2031.03 | 1747.747 | 1661.274 |
|  | Скорость обучения | 639 ms | 1.99 s | 22.2 s | 
|  | Скорость предсказания | 13 ms | 423 ms | 161 ms | 
| Вариант № 2 | RMSE | 2043.876 | 1720.709 | 1705.446 |
|  | Скорость обучения | 1.05 s | 2.13 s | 58.5 s | 
|  | Скорость предсказания | 28.9 ms | 445 ms | 517 ms | 

Как видно из итоговой таблицы, дерево решений с увеличением глубины обучения переобучилось и стало плохо предсказывать. То же самое произошло и с моделью CatBoostRegressor - ее время обучения и предсказания увеличилось, а ключевая метрика ухудшилась. LGBMRegressor с увеличение глубины улучшил свои показатели, но ненамного.

## Проверка лучшей модели на test выборке

In [43]:
%%time

predictions_test = model_CB.predict(features_test)
rmse = mean_squared_error(target_test, predictions_test) ** .5
display(rmse)

1662.8302758550008

Wall time: 166 ms


RMSE на тестовой выборке составляет 1662.83$

## Общий вывод

Исходя из проведенного анализа данных и построения модели машинного обучения можно сделать вывод, что в соответствии с критериями, установленными заказчиком, а именно:
 * качество предсказания;
 * время обучения модели;
 * время предсказания модели.
 
самой подходящей моделью является первый вариант CatBoostRegressor с max_depth=5, т.к. скорость ее предсказания достаточно высока при низкой RMSE. Обучение модели проходит примерно в 4 раза дольше, чем обучение LGBMRegressor, однако, т.к. данные статичные, то переобучение модели можно производить нединамично и эта метрика менее важная, чем отлик модели на запрос пользователя. RMSE на тестовой выборке показывает значение 1662.83$, что несильно отличается от показателя на валидационной выборке.